In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from openai import OpenAI
import feedparser
import time
import json

In [9]:
feed_url = 'https://pythoninvest.com/rss-feed-612566707351.xml'
output_file_path = 'data/input_news_feed.json'

feed = feedparser.parse(feed_url)

In [10]:
print(f'Length of feed entries: {len(feed['entries'])}')

Length of feed entries: 49


In [11]:
print(f'Keys of one feed entry: {feed['entries'][0].keys()}')

Keys of one feed entry: dict_keys(['title', 'title_detail', 'links', 'link', 'published', 'published_parsed', 'authors', 'author', 'author_detail', 'tags', 'summary', 'summary_detail', 'turbo_content'])


In [12]:

# If the RSS feed format would be same , then the input to the method would be an element of the python dict `feed`. 
# So input to this method would be an element which is of the type `dict`.
def extract_turbo_content(entry_element):
    """
    Extracts turbo:content from the raw XML
    """
    try:
        if entry_element:  
            content = entry_element.get('turbo_content')  
        return content if content else None

    except (AttributeError, KeyError, TypeError) as e:
        # AttributeError: If entry_element doesn't support dictionary operations
        # KeyError: If 'turbo_content' doesn't exist
        # TypeError: If entry_element is not of the expected type
        print(f"Error extracting turbo content: {str(e)}")

In [13]:
feed['entries'][0]['link']


'https://pythoninvest.com/tpost/yk09rupzv1-week-17-24-july-2023'

In [14]:
feed['entries'][0]['turbo_content']

'<header><h1>Week 17-24 July 2023</h1></header><figure><img src="https://static.tildacdn.com/tild6131-3163-4135-a565-356661323733/the-broad-industry-o.svg"/></figure><div class="t-redactor__embedcode"><script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-5845276189467216"\n     crossorigin="anonymous"></script></div><div class="t-redactor__text"><strong>DISCLAIMER</strong><br />The content provided below was generated by AI (OpenAI\'s ChatGPT) using titles and descriptions from a selection of 5000 published financial news articles. The information presented should be used for informational purposes only and does not constitute financial advice or investment recommendations. We advise readers to conduct their own research and consult with a qualified financial advisor before making any investment decisions. The AI-generated content may not reflect the most current market conditions or developments and should be considered as a general summary of

In [15]:
prompt_template = '''Expert Web Scraper.
HTML Content: {content}

Perform different types of text extraction:
1) Extract individual news text AS IT IS from given HTML.
HTML Content format:
INDIVIDUAL NEWS SUMMARY
Start date for the articles: <start_date>; End date for the articles: <end_date>
NEWS SUMMARY for (<ticker>, <count>), which changed on <growth>% last trading day:
<text?

You need to extract all fields in <> :
- Date ranges
- mentioned ticker 
- news count
- growth percentage
- news for the ticker

Format:
{{
  "content": [
    {{
      "type": "individual",
      "start_date": <start date for articles>,
      "end_date": <end date for articles>,
      "ticker": <ticker symbol from news>,
      "count": <articles count from news>,
      "growth": <growth %>
      "text": <news for the ticker from html>,
    }},
    // repeat for all news
  ]
}}

2) Extract market news 1 day or 1 week text AS IT IS from given HTML:
HTML Content format:
[<model_name> <period> summary] MARKET NEWS SUMMARY ('multiple_tickers', <news_count> ) -- i.e. <news_count> news summary for the last 24 hours before <end_date> UTC time:

Extract text AS IT IS from given HTML:
- <model_name>
- <period>
- <news_count>
- <news_summary>


Output JSON format:
{{
  "content": [
    {{
      "type": "market_"+<period>,
      "end_date": <end_date>,
      "start_date": <24 hours before end_date>,
      "ticker": "multiple_tickers",
      "count": <news_count>,
      "model": <model_name>
      "text": <news_summary>,
    }},
  ]
}}


Constraints:
Return JSON only. 
'''